In [1]:
import ambit
import numpy as np
import itertools as it
ambit.initialize()

In [2]:
# Tuneage, will move later
tensor_type = ambit.TensorType.CoreTensor
max_test_rank = 4

In [3]:
def build(name, dims, ttype=tensor_type, fill=None):
    # Builds arbitrary Tensors and numpy arrys
    # Accepts a list of integers or a string for dimensions

    if ttype != ambit.TensorType.CoreTensor:
        raise ValueError("Only CoreTensor is currently supported")

    T = ambit.Tensor.build(ttype, name, dims)

    # Fill both N and T
    N = np.asarray(T)

    N.flat[:] = fill

    # Copy numpy array so we no longer share memory
    N = N.copy()

    return [T, N]

In [43]:
dim = 10000
ambit_A, numpy_A = build("A", [dim,dim], fill=np.random.rand(dim,dim))
ambit_B, numpy_B = build("B", [dim,dim], fill=np.random.rand(dim,dim))
ambit_C, numpy_C = build("C", [dim,dim], fill=np.zeros((dim,dim)))

In [39]:
%time ambit_C["ij"] = ambit_A["ik"] * ambit_B["kj"]

CPU times: user 1min 46s, sys: 10.8 s, total: 1min 57s
Wall time: 7.36 s


In [40]:
%time numpy_C = numpy_A @ numpy_B

CPU times: user 1min 14s, sys: 1.46 s, total: 1min 15s
Wall time: 9.47 s


In [41]:
np.allclose(ambit_C, numpy_C)

True

In [11]:
dim = 100
ambit_A, numpy_A = build("A", [dim,dim,dim,dim], fill=np.random.rand(dim,dim,dim,dim))
ambit_B, numpy_B = build("B", [dim,dim,dim,dim], fill=np.random.rand(dim,dim,dim,dim))
ambit_C, numpy_C = build("C", [dim,dim,dim,dim], fill=np.zeros((dim,dim,dim,dim)))

In [12]:
%time ambit_C["ijmn"] = ambit_A["ijkl"] * ambit_B["klmn"]

CPU times: user 1min 44s, sys: 8.05 s, total: 1min 52s
Wall time: 7.04 s


In [13]:
%time numpy_C = np.einsum("ijkl,klmn->ijmn", numpy_A, numpy_B, optimize=True)

CPU times: user 1min 13s, sys: 244 ms, total: 1min 13s
Wall time: 9.38 s


In [14]:
np.allclose(ambit_C, numpy_C)

True